In [1]:
import re
import os
import cohere
import requests

import numpy as np
import pandas as pd
import plotly.express as px

from lxml import html
from bs4 import BeautifulSoup
from unidecode import unidecode

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, rand_score

In [2]:
artist = "lorde"
API_KEY = "YOUR_API_KEY"

In [3]:
class Discography:
    def __init__(self, artist, albums, verbose=True):
        self.verbose = verbose
        self.artist = artist
        self.albums = albums
        self.num_albums = len(albums)
        self.co = cohere.Client(API_KEY)
        self.df = self.get_df()

    def get_df(self) -> pd.DataFrame:
        """Get pandas dataframe from albums - each row is a song
        
        Returns:
            pd.DataFrame -- dataframe with columns: artist, album, song_title, lyrics, url, lyrics_embedding
        """
        if self.verbose:
            print(f"Getting dataframe for {self.artist}...")

        df = pd.DataFrame()
        for album in self.albums:
            df = pd.concat(
                [
                    df,
                    pd.DataFrame(
                        {
                            "artist": self.artist,
                            "album": album.title,
                            "song_title": [song.title for song in album.songs],
                            "lyrics": [song.lyrics for song in album.songs],
                            "url": [song.url for song in album.songs],
                        }
                    ),
                ]
            )

        df = df.reset_index(drop=True)
        df = self.get_embeddings(df, "lyrics")
        df = self.get_embeddings(df, "song_title")

        return df

    def get_embeddings(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Get embeddings for a column
        
        Args:
            df (pd.DataFrame): dataframe
            column (str): column to get embeddings for

        Returns:
            pd.DataFrame -- dataframe with column: {column}_embedding
        """

        if self.verbose:
            print(f"Getting embeddings for {column}...")
        
        texts = df[column].tolist()
        response = self.co.embed(model="large", texts=texts)
        df[f"{column}_embedding"] = response.embeddings
        return df

In [4]:
class Album:
    """Album class - Album wrapper

    Attributes:
        title (str): Album title
        songs (list): List of Song objects
    """

    def __init__(self, title, songs):
        self.title = title
        self.songs = songs

In [5]:
class Song:
    """Song class - Song wrapper

    Attributes:
        title (str): Song title
        lyrics (str): Song lyrics
        url (str): Song url
    """

    def __init__(self, title: str, lyrics: str, url: str):
        self.title = title
        self.lyrics = lyrics
        self.url = url

In [6]:
class Scraper:
    """Scraper class - Get albums and songs from artist page

    Attributes:
        artist (str): Artist name
        max_albums (int): Max number of albums to get
    """

    def __init__(self, artist, max_albums=100, verbose=False):
        self.artist = artist
        self.max_albums = max_albums
        self.verbose = verbose
        self.main_url = "https://www.letras.mus.br"
        self.clean_artist = artist.lower().replace(" ", "-")
        self.url = self.main_url + "/" + self.clean_artist + "/discografia"

    def clean_album_title(self, album_title: str) -> str:
        """Clean album title by removing anything in parenthesis - like (Deluxe Edition)
        
        Args:
            album_title (str): Album title
        
        Returns:
            str: Cleaned album title
        """
        album_title = re.sub(r"\([^)]*\)", "", album_title)
        return unidecode(album_title).lower().strip()

    def make_request(self, url: str) -> requests.models.Response:
        """Make request with the request module and treat errors
        
        Args:
            url (str): Url to make request

        Returns:
            requests.models.Response: Response object
        """

        headers = {
            "sec-ch-ua": '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
            "sec-ch-ua-mobile": "?0",
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36",
        }

        try:
            response = requests.get(url, headers=headers)
        except Exception as e:
            print(f"Error - {e.args}")
            return None

        if response.status_code != 200:
            return None

        return response

    def get_discography(self) -> Discography:
        """Get all albums from artist page
        
        Returns:
            Discography: Discography object
        """

        # get html from artist page - letras.mus.br
        response = self.make_request(self.url)
        if response is None:
            return None

        soup = BeautifulSoup(response.text, "lxml")
        albums_tags = soup.find_all("div", class_="album-item g-sp")

        albums = []
        albuns_clean_names = []
        # iterate over albums reversed to exclude albuns like deluxe and live versions
        for i, album_tag in enumerate(albums_tags[::-1], 1):
            album_title = album_tag.find("h1").text.strip()
            info_type = album_tag.find("span", class_="header-info-type").text

            if self.verbose:
                print(f"[{i}/{len(albums_tags)}] - {album_title}")

            # excludes singles and EPs
            if "Álbum" not in info_type:
                if self.verbose:
                    print("\tNot an album")
                continue

            curr_album_clean_name = self.clean_album_title(album_title)

            # excludes albums with the same name
            if curr_album_clean_name in albuns_clean_names:
                if self.verbose:
                    print("\tAlbum already exists")
                continue
            else:
                albuns_clean_names.append(curr_album_clean_name)

            songs_tags = album_tag.find_all("a", class_="bt-play-song")

            songs = []
            for j, song_tag in enumerate(songs_tags, 1):
                if self.verbose:
                    print(f"\t[{j}/{len(songs_tags)}] - ", end="")
                
                url = self.main_url + song_tag.get("href")
                response = self.make_request(url)
                
                if response is None:
                    if self.verbose:
                        print("Failed")
                    continue

                tree = html.fromstring(response.content)

                song_title = tree.xpath("//div[@class='cnt-head_title']/h1")[0].text
                
                if self.verbose:
                    print(song_title)

                # get lyrics
                try:
                    lyrics = tree.xpath('//div[@class="cnt-letra"]//text()')
                    lyrics = "\n".join(lyrics).strip()

                    if lyrics == "":
                        raise Exception(f"No lyrics {url}")

                    # if lyrics is empty this is the default message
                    if "Ainda nÃ£o temos a letra desta mÃºsica." in lyrics:
                        raise Exception("No lyrics")
                except Exception as e:
                    if self.verbose:
                        print(f"\t\tCouldn't get lyrics {url}")
                    continue

                songs.append(Song(song_title, lyrics, url))

            albums.append(Album(album_title, songs))

            # break if max albums is reached
            if len(albums) >= self.max_albums:
                break

        discography = Discography(self.artist, albums)
        return discography

In [7]:
class Plot:
    """Plot class - Plot embeddings and confusion matrix

    Attributes:
        discography (Discography): Discography object
        column (str): Column to plot (lyrics or song_title)
    """

    def __init__(self, discography: Discography, column: str = "lyrics"):
        if column not in ["lyrics", "song_title"]:
            raise ValueError("Column must be 'lyrics' or 'song_title'")
        
        self.column = column
        self.discography = discography
        self.artist = discography.artist
        self.df = discography.df
        self.set_up()

    def set_up(self):
        """Set up embeddings and clusters"""
        # Reduce dimensionality with PCA
        tsne = TSNE(n_components=2, random_state=0, perplexity=5)
        embeddings_column = f'{self.column}_embedding'
        self.embeddings = self.df[embeddings_column].tolist()
        self.embeddings_2d = tsne.fit_transform(np.array(self.embeddings))

        # Cluster embeddings
        num_clusters = self.discography.num_albums
        self.kmeans = KMeans(n_clusters=num_clusters, random_state=0)
        self.clusters = self.kmeans.fit_predict(self.embeddings)

        self.df['cluster'] = self.clusters

        self.df['embeddings_x'] = self.embeddings_2d[:, 0]
        self.df['embeddings_y'] = self.embeddings_2d[:, 1]

    def plot_clusters(self, download: bool = False, path: str = 'imgs'):
        """Plot clusters
        
        Args:
            download (bool, optional): Wheter to download clusters. Defaults to False.
        """
        
        self.df['album_label'] = self.df['album'].astype('category').cat.codes
        self.df['size'] = 1

        # Create scatter plot
        fig = px.scatter_3d(self.df, x='embeddings_x', y='embeddings_y', z='cluster', color='album',
                        hover_name='album', hover_data=['album', 'song_title'], 
                        size='size', size_max=10, title=f'Song coordinates in 2D space - {self.artist} {self.column}', width=1200, height=800)

        fig.update_traces(hovertemplate='<b>%{hovertext}</b><br>' +
                                        'Album: %{customdata[0]}<br>' +
                                        'Cluster: %{z}<br>' +
                                        'Song Title: %{customdata[1]}<br>'+
                                        'x: %{x}<br>' +
                                        'y: %{y}<br>' +
                                        'z: %{z}<br><extra></extra>'
                                        )

        
        if download:
            if path is None:
                path = ''
            elif not os.path.exists(path):
                os.makedirs(path)

            fig.write_image(f'{path}/scatter_plot_{self.artist}_{self.column}.png')
        
        counts = self.df.value_counts(['cluster', 'album']).sort_index()
        pd.set_option('display.max_rows', None)
        print(counts)
        fig.show()
    
    def get_rand_score(self, download: bool = False):
        """Get rand score which measures how many pair of objects are in the same cluster in the data and in the same cluster in the prediction"""
        score = rand_score(self.df['album'], self.df['cluster'])
        print(f"Rand score: {score}")

    def plot_sillhouette_score(self, download: bool = False, path: str = 'imgs'):
        """Plot the average silhouette score for each album which measures how similar an object is to its own cluster
        
        Args:
            download (bool, optional): Wheter to download silhouette score. Defaults to False.
        """
        album_names = self.df['album'].unique()
        album_scores = {}
        for album in album_names:
            embeddings = self.df[self.df['album'] == album][f'{self.column}_embedding'].tolist()
            clusters = self.df[self.df['album'] == album]['cluster'].tolist()

            try:
                silhouette_avg = silhouette_score(embeddings, clusters, metric='cosine')
            except ValueError:
                print(f"ValueError: Only one data point for album {album} - need more to calculate the silhouette score")
                continue
            
            album_scores[album] = silhouette_avg

        df_scores = pd.DataFrame(list(album_scores.items()), columns=['Album', 'Silhouette Score'])

        fig = px.bar(df_scores, x='Silhouette Score', y='Album', 
                    hover_data={'Silhouette Score': ':.3f'},
                    text='Silhouette Score', 
                    color='Silhouette Score',
                    labels={'Silhouette Score': 'Silhouette Score', 'Album': 'Album'},
                    title=f'Avg Silhouette Score for Each Album - {self.artist} {self.column}')

        if download:
            if path is None:
                path = ''
            elif not os.path.exists(path):
                os.makedirs(path)
            fig.write_image(f'{path}/sillhouette_plot_{self.artist}_{self.column}.png')

        fig.show()


In [8]:
scraper = Scraper(artist, max_albums=20, verbose=True)
discography = scraper.get_discography()

[1/28] - Royals
	Not an album
[2/28] - Pure Heroine
	[1/10] - Tennis Court
	[2/10] - 400 Lux
	[3/10] - Royals
	[4/10] - Ribs
	[5/10] - Buzzcut Season
	[6/10] - Team
	[7/10] - Glory And Gore
	[8/10] - Still Sane
	[9/10] - White Teeth Teens
	[10/10] - A World Alone
[3/28] - Pure Heroine (Extended)
	Album already exists
[4/28] - Live In Concert
	Not an album
[5/28] - No Better
	Not an album
[6/28] - Team
	Not an album
[7/28] - Ribs
	Not an album
[8/28] - Glory And Gore
	Not an album
[9/28] - Buzzcut Season
	Not an album
[10/28] - The Love Club
	Not an album
[11/28] - Tennis Court EP
	Not an album
[12/28] - Flicker (Kanye West Rework)
	Not an album
[13/28] - Yellow Flicker Beat
	Not an album
[14/28] - Magnets
	Not an album
[15/28] - Melodrama
	[1/12] - Green Light
	[2/12] - Sober
	[3/12] - Homemade Dynamite
	[4/12] - The Louvre
	[5/12] - Liability
	[6/12] - Hard Feelings / Loveless
	[7/12] - Sober II (Melodrama)
	[8/12] - Writer In The Dark
	[9/12] - Supercut
	[10/12] - Liability (Reprise)

## Lyrics embeddings

In [9]:
pt = Plot(discography)

/Users/luisa/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [10]:
pt.plot_sillhouette_score()

In [11]:
pt.plot_clusters()

cluster  album       
0        Melodrama        1
         Pure Heroine     1
         Solar Power      8
1        Melodrama        1
         Pure Heroine     6
         Solar Power      2
2        Melodrama       10
         Pure Heroine     3
         Solar Power      2
dtype: int64


In [12]:
pt.get_rand_score()

Rand score: 0.6844919786096256


## Song titles embeddings

In [13]:
pt = Plot(discography, column="song_title")

/Users/luisa/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [14]:
pt.plot_clusters()

cluster  album       
0        Melodrama       2
         Pure Heroine    3
         Solar Power     1
1        Melodrama       7
         Pure Heroine    3
         Solar Power     5
2        Melodrama       3
         Pure Heroine    4
         Solar Power     6
dtype: int64


In [15]:
pt.plot_sillhouette_score()

In [16]:
pt.get_rand_score()

Rand score: 0.5525846702317291
